In [1]:
#read the data in to inspect it
with open ('/Users/SHSU/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/Springboard/dsc/Capstone Three/HC_DATA/medquad_qa_pairs.csv', 'r') as f:
    df = f.read()

In [2]:
print('length of dataset in characters:', len(df))

length of dataset in characters: 22835185


In [6]:
print(df[:1000])

question,answer,source,focus_area
What is (are) Glaucoma ?,"Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vis

In [8]:
chars = sorted(list(set(df)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{}~®°µ¼éñīō–—’“”…−≥
111


In [10]:
stoi ={ch:i for i, ch in enumerate (chars) }
itos ={i:ch for i, ch in enumerate (chars) }
encode = lambda s: [stoi[c] for c in s] # encoder : take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decpder: take a lost of integers, output a string

print(encode("hii there"))
print(decode (encode('hii there')))

[73, 74, 74, 2, 85, 73, 70, 83, 70]
hii there


In [15]:
#let's now encode the entire text dataset and store it into a torch.tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(df), dtype= torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characers we lloked at earlier will to the GPT look like this.

torch.Size([22835185]) torch.int64
tensor([82, 86, 70, 84, 85, 74, 80, 79, 14, 66, 79, 84, 88, 70, 83, 14, 84, 80,
        86, 83, 68, 70, 14, 71, 80, 68, 86, 84, 64, 66, 83, 70, 66,  1, 57, 73,
        66, 85,  2, 74, 84,  2, 10, 66, 83, 70, 11,  2, 41, 77, 66, 86, 68, 80,
        78, 66,  2, 33, 14,  4, 41, 77, 66, 86, 68, 80, 78, 66,  2, 74, 84,  2,
        66,  2, 72, 83, 80, 86, 81,  2, 80, 71,  2, 69, 74, 84, 70, 66, 84, 70,
        84,  2, 85, 73, 66, 85,  2, 68, 66, 79,  2, 69, 66, 78, 66, 72, 70,  2,
        85, 73, 70,  2, 70, 90, 70,  9, 84,  2, 80, 81, 85, 74, 68,  2, 79, 70,
        83, 87, 70,  2, 66, 79, 69,  2, 83, 70, 84, 86, 77, 85,  2, 74, 79,  2,
        87, 74, 84, 74, 80, 79,  2, 77, 80, 84, 84,  2, 66, 79, 69,  2, 67, 77,
        74, 79, 69, 79, 70, 84, 84, 16,  2, 57, 73, 74, 77, 70,  2, 72, 77, 66,
        86, 68, 80, 78, 66,  2, 68, 66, 79,  2, 84, 85, 83, 74, 76, 70,  2, 66,
        79, 90, 80, 79, 70, 14,  2, 85, 73, 70,  2, 83, 74, 84, 76,  2, 74, 84,
     

In [16]:
#let's now split up the data into train and validation sets
n= int(0.9 * len(data)) #first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [17]:
block_size = 8
train_data [:block_size + 1]

tensor([82, 86, 70, 84, 85, 74, 80, 79, 14])

In [20]:
x= train_data[:block_size]
y= train_data[1:block_size + 1]
for t in range (block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([82]) the target: 86
when input is tensor([82, 86]) the target: 70
when input is tensor([82, 86, 70]) the target: 84
when input is tensor([82, 86, 70, 84]) the target: 85
when input is tensor([82, 86, 70, 84, 85]) the target: 74
when input is tensor([82, 86, 70, 84, 85, 74]) the target: 80
when input is tensor([82, 86, 70, 84, 85, 74, 80]) the target: 79
when input is tensor([82, 86, 70, 84, 85, 74, 80, 79]) the target: 14


In [24]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we procss in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split): 
    #generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x= torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): #time dimension
        context= xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target:{target}')

inputs:
torch.Size([4, 8])
tensor([[69, 74, 66, 72, 79, 80, 84, 74],
        [84, 85, 80, 81,  2, 68, 66, 79],
        [15,  2,  2, 39, 89, 71, 80, 77],
        [80, 78, 70,  2, 84, 86, 68, 68]])
targets:
torch.Size([4, 8])
tensor([[74, 66, 72, 79, 80, 84, 74, 84],
        [85, 80, 81,  2, 68, 66, 79, 68],
        [ 2,  2, 39, 89, 71, 80, 77, 74],
        [78, 70,  2, 84, 86, 68, 68, 70]])
-----
when input is [69] the target:74
when input is [69, 74] the target:66
when input is [69, 74, 66] the target:72
when input is [69, 74, 66, 72] the target:79
when input is [69, 74, 66, 72, 79] the target:80
when input is [69, 74, 66, 72, 79, 80] the target:84
when input is [69, 74, 66, 72, 79, 80, 84] the target:74
when input is [69, 74, 66, 72, 79, 80, 84, 74] the target:84
when input is [84] the target:85
when input is [84, 85] the target:80
when input is [84, 85, 80] the target:81
when input is [84, 85, 80, 81] the target:2
when input is [84, 85, 80, 81, 2] the target:68
when input is [84, 85,

In [25]:
print(xb) # our input to the transformer

tensor([[69, 74, 66, 72, 79, 80, 84, 74],
        [84, 85, 80, 81,  2, 68, 66, 79],
        [15,  2,  2, 39, 89, 71, 80, 77],
        [80, 78, 70,  2, 84, 86, 68, 68]])


In [49]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next roken from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
                B, T, C = logits.shape
                logits = logits.view(B*T, C)
                targets = targets.view(B*T)
                loss = F.cross_entropy(logits, targets)


        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax (logits, dim=-1) #(B,C)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples =1) # (B,1)
            #append sampled index to the running sequence 
            idx=torch.cat((idx, idx_next), dim=1) # (B,T+1)
        return idx

m= BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
out = m(xb, yb)
print (logits.shape)
print(loss)

idx= torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx= torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 111])
tensor(5.0754, grad_fn=<NllLossBackward0>)
	<−o4®≥uPK(}L)"=y+Cc"[mzAōf¼yt?YZ-[`—SS6#FH®C8<Rs{\ZOK’j5ETp&A0g5M;2ñsRt“8j—*Ze30…}qs&rRD@r?’9`=−b!$g


In [50]:
# create a PyTorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [60]:
batch_size =32
for steps in range (10000):
    # sample a batch of data
    xb, yb = get_batch ('train')

    #evaluate the loss
    logits, loss= m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5620908737182617


In [65]:
print(decode(m.generate(idx= torch.zeros((1,1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))

	 titontacivepanth  onche Hy orarass al  nomasivew  e  t nc tear?,Gicot fe; ur wartronge  m tig : w:
Whalpins we bns- prympelofitew (an thes it,Latsinty ple nopasther itig). mastrmart y iss pe. cr ?"Jal cherercofialodonep celoncesclynf ve lio Thericof thes Ininind  anse ss inded gerop  athestent f   oms tithre imus  thecoumalo plevathan iofiof turshiatorom  ompstis, avacie   orn be toth, hyogess  ung.  oneajus aualt ecatt find edenoneen ierowe corex. s Mondr o cstes, origngese ty mpequs suinentr reg, are ictutwangedis.", an canererciourelegr ie l, - he  ube muionl gyphe mf mais lsugives nd to a th ms id trrd ty al dele rurmoworof KSof  (ed  ongy    HSymp tol 
 wa actheas aben sly  ifose angiowalere-cende ar sm ofoing s,"Syecore alon mo t weng fumata Sjus pr thalvePamayma--s ag.
 thalesinict tharor ?,GIf (aberealee ogoment- brethefotin,",  atinenflior cel Thar, anomaly  ifiatwithaf  tignjomperomatodovealaind cidettile. acofechineldounnathesse
MPAasun ithefisigepithy cend bnbectallveovio

###The mathematical trick in self attention###

In [66]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2 # batch, time, channels
x= torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [67]:
#we want x[b,t] = mean_{i<=t} x[b,i]
xbow=torch.zeros((B,T,C))
for b in range (B):
    for t in range (T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [76]:
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True)
xbow2= wei @ x #(t,T) @ (B, T, C) -------> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [80]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
xbow3 = wei @ x 
torch.allclose(xbow, xbow3)

False

In [74]:
torch.manual_seed(42)
a= torch.tril(torch.ones(3,3))
a = a /torch.sum(a, 1, keepdim=True)
b= torch.randint(0,10,(3,2)).float()
c= a@b
print('a=')
print(a)
print('----')
print('b=')
print(b)
print('----')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [82]:
#version 4: self-attention !
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x=torch.randn(B,T,C)

tril = torch.tril(torch.ones(T,T))
wei= torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei=F.softmax(wei, dim = -1)
out = wei @ x
out.shape


torch.Size([4, 8, 32])

In [83]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [84]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])